In [ ]:

import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch

In [ ]:
def f(x):
  return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:

xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
from IPython import display
display.Image(url = 'https://wikimedia.org/api/rest_v1/media/math/render/svg/aae79a56cdcbc44af1612a50f06169b07f02cbf3')

In [ ]:
h = 0.000001
x = 2/3
(f(x + h) - f(x))/h

In [ ]:
# let's make it more complex
 
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
h = 0.0001

# inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
c += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1)/h)

In [ ]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad 
      other.grad += 1.0 * out.grad
    out._backward = _backward
    # the old version is  self.grad = 1.0 * out.grad. But If the same value is used twice in an equation grad values will overwrited
    # So in this bug grad values are not updated. We wanna calculate grad values cumulative way so new equation is self.grad += 1.0 * out.grad
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def backward(self): # automatized backward function
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
draw_dot(L)

In [ ]:
def check_grad():
  
  h = 0.001
  
  # base
  a = Value(2.0, label='a')
  b = Value(-3.0, label='b')
  c = Value(10.0, label='c')
  e = a*b; e.label = 'e'
  d = e + c; d.label = 'd'
  f = Value(-2.0, label='f')
  L = d * f; L.label = 'L'
  L1 = L.data
  
  # control group
  a = Value(2.0, label='a')
  b = Value(-3.0, label='b')
  b.data += h
  c = Value(10.0, label='c')
  e = a*b; e.label = 'e'
  d = e + c; d.label = 'd'
  f = Value(-2.0, label='f')
  L = d * f; L.label = 'L'
  L2 = L.data
  
  
  print((L2 - L1)/h)
  
# calculate grad for b

check_grad()

* The gradient value tells us how much and in which direction we should change the weight value.

In [ ]:
# use check_grad function (also you can use derivative formul for being sure) for find and assign grad value to node
L.grad = 1
f.grad = 4
d.grad = -2
e.grad = -2
c.grad = -2
b.grad = -4
a.grad = 6

In [ ]:
draw_dot(L)

* Calculate grad value for some nodes with manuel method
* * a = 2, b = -3, c=10, e = axb, d = e+c, f = -2, L = dxf
* * ...



In [ ]:
a.data += 0.01 * a.grad # value update with 0.01 learning rate backforward
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a * b # forward pass
d = e + c
L = d * f

print(L.data)

# manuel backpropagation on neuron

In [ ]:
from IPython import display
display.Image(url = 'https://cs231n.github.io/assets/nn1/neuron_model.jpeg')

In [ ]:
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid(); # tanh function

* we are using tanh function as a activation function because we need the differentiable function for getting derivative of output node at backpropagation step
* so activation function must be differentiable for model's learning

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'nn'
o = n.tanh(); o.label = 'output'

In [ ]:
draw_dot(o)

In [ ]:
o.grad = 1 # this is a default

In [ ]:
draw_dot(o)

In [ ]:
from IPython import display
display.Image(url = 'https://wikimedia.org/api/rest_v1/media/math/render/svg/48e115d0848c424686f85f8d33cfe9f81c95cd58')

* so we can use tanh derivative 1-tanh**2(x)
* output = tanh(nn)
    ---> d(output)/ d(nn) = 1- tanh(nn)**2 ---> 1- o **2 


In [ ]:
1 - o.data**2 # so this is a derivative of output it means grad of n

In [ ]:
n.grad = 0.5

In [ ]:
draw_dot(o)

*  plus is just a distributor of gradient so b and x1w1x2w2 nodes get the nn's gradient because of derivative rules

In [ ]:
b.grad = n.grad
x1w1x2w2.grad = n.grad

In [ ]:
draw_dot(o)

* we have a another plus step so appling same rule

In [ ]:
x1w1.grad = x1w1x2w2.grad
x2w2.grad = x1w1x2w2.grad

In [ ]:
draw_dot(o)

* now we have a multiplication process x1 * w1 = x1w1
* in multiplication process we use the grad of result node on derivative
*   * x1w1  = x1 * w1  ---> d(x1w1) / d(x1) = w1 * x1w1.grad (for x1 derivative)
    * x1w1 = x1 * w1   ---> d(x1w1) / d(w1) = x1 * x1w1.grad (for w1 derivative)

In [ ]:
x1.grad = w1.data * x1w1.grad
w1.grad = x1.data * x1w1.grad

In [ ]:
draw_dot(o)

* same situtiaon on x2w2 = x2 * w2
* in multiplication process we use the grad of result node on derivative
*   * x2w2  = x2 * w2  ---> d(x2w2) / d(x2) = w2 * x2w2.grad (for x2 derivative)
    * x2w2 = x2 * w2   ---> d(x2w2) / d(w2) = x2 * x2w2.grad (for w2 derivative)


In [ ]:
w2.grad = x2.data * x2w2.grad
x2.grad = w2.data * x2w2.grad

In [ ]:
draw_dot(o)

# backpropagation with function

* we applied manuel backpropagation on above code, now we define the function in value  class  for doing automaticly (_backwards functions)
* now lets try with function

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'nn'
o = n.tanh(); o.label = 'output'

In [ ]:
draw_dot(o)

In [ ]:
o.grad = 1 # default assign coming from derivative rule
draw_dot(o)

In [ ]:
o._backward()
draw_dot(o)

In [ ]:
n._backward()
draw_dot(o)

In [ ]:
x1w1x2w2._backward()
draw_dot(o)

In [ ]:
x1w1._backward()
draw_dot(o)

In [ ]:
x2w2._backward()
draw_dot(o)

* Now We want to make a function that will do all the update operations. We need the topological sort because all nodes connected with each other
* First of all we need set grad values to 0 for observation

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'nn'
o = n.tanh(); o.label = 'output'

In [ ]:
draw_dot(o)

In [ ]:
topo = []
visited = set()
def build_topo(v):
  if v not in visited:
    visited.add(v)
    for child in v._prev:
      build_topo(child)
    topo.append(v)
build_topo(o)
topo

In [ ]:
o.grad = 1

topo = []
visited = set()
def build_topo(v):
  if v not in visited:
    visited.add(v)
    for child in v._prev:
      build_topo(child)
    topo.append(v)
build_topo(o)


for node in reversed(topo):
  node._backward()
# we are going to hiding this function to Value class

In [ ]:
draw_dot(o)

* now we gonna use backward function that exist in Value class

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'nn'
o = n.tanh(); o.label = 'output'

In [ ]:
draw_dot(o)

In [ ]:
o.backward()

In [ ]:
draw_dot(o)

* there was a bug in old version code lets check it out

In [ ]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad = 1.0 * out.grad   # BUG IN HERE!!!!!
      other.grad = 1.0 * out.grad # BUG IN HERE!!!!!
    out._backward = _backward

    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def backward(self): # automatized backward function
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

In [ ]:
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'
b.backward()
draw_dot(b)

* When we used the same value on add operation we encountered this bug. Because the _backward() function in Value class its wrong
* self.grad = 1.0 * out.grad . In this case we are overwriting to grad value so it has been not  updated
* In this example node a's grad value should be 2 not 1.


In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b    ; d.label = 'd'
e = a + b    ; e.label = 'e'
f = d * e    ; f.label = 'f'

f.backward()

draw_dot(f)

* Different version for this bug
* So we dont want overwrite to grad value. We want to cumulative update.

In [ ]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad   # BUG FIXED!!!!!
      other.grad += 1.0 * out.grad #  BUG FIXED!!!!!
    out._backward = _backward

    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def backward(self): # automatized backward function
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

In [ ]:
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'
b.backward()
draw_dot(b)

In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b    ; d.label = 'd'
e = a + b    ; e.label = 'e'
f = d * e    ; f.label = 'f'

f.backward()

draw_dot(f)

* Now bug fixed...

* In the example above, we used one of the derivative versions of the tanh function.
* now we use different derivative version of tanh function ant lets check result that we will find is equal old result? 
* First of all we have to update Value class for geting derivative different version of tanh


In [ ]:
from IPython import display
display.Image(url = 'https://wikimedia.org/api/rest_v1/media/math/render/svg/3f2cf2340523c38532abdb7103962b9f990701d5')

In [ ]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

 * Import notes  !!!

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()

* with first version of tanh derivative

In [ ]:
draw_dot(o)

* with 2. version of derivative tanh

In [ ]:

# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# ----
e = (2*n).exp()
o = (e - 1) / (e + 1)
# ----
o.label = 'o'
o.backward()
draw_dot(o)

# torch part

In [ ]:

import torch

In [ ]:
torch.Tensor([2.0])

In [ ]:
torch.Tensor([2.0]).dtype # default type

In [ ]:
torch.Tensor([2.0]).double().dtype

* Let's do the apply we did create backpropagation manually above with pytroch.

In [ ]:
x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True 
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True 
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

* pytorch assumes they do not require gradients so we need to say that all of the these nodes require gradients

In [ ]:
o

In [ ]:
o.item()

In [ ]:
import random

In [ ]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
  # w*x +b
    print(list(zip(self.w, x))) # for checkin
    act_func = sum((wi*xi for wi, xi in zip(self.w , x)),self.b) 
    out = act_func.tanh()
    return out

x = [2.0, 3.0]
n = Neuron(2)
n(x)
  

  
 
  

* now we can forward single neuron
* now lets start define the layers of neuron

In [ ]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
  # w*x +b
    #print(list(zip(self.w, x))) # for checkin
    act_func = sum((wi*xi for wi, xi in zip(self.w , x)),self.b) 
    out = act_func.tanh()
    return out

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
       outs = [n(x) for n in self.neurons]
       return outs

x = [2.0, 3.0] 
n = Layer(2,3) # 2 dimensional neuron 
n(x)

* lets build mlp version

In [ ]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
  # w*x +b
    #print(list(zip(self.w, x))) # for checkin
    act_func = sum((wi*xi for wi, xi in zip(self.w , x)),self.b) 
    out = act_func.tanh()
    return out

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
       outs = [n(x) for n in self.neurons]
       return outs[0] if len(outs)==1 else outs
    

    
class MLP:
  def __init__(self, nin, nouts): # nouts = number of neuron on single layer, nin = number of inputs
    sz = [nin] + nouts # sz= size
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))] # create layer object
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x







x = [2.0, 3.0, -1.0] # input neuron's data
n = MLP(3, [4,4,1]) 
n(x)

In [ ]:
from IPython import display
display.Image(url = 'https://cs231n.github.io/assets/nn1/neural_net2.jpeg')

* This is the model we built in the code above

In [ ]:
draw_dot(n(x))

* now we gonna apply backpropagation on dummy dataset

In [ ]:

xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred  = [n(x) for x in xs]
ypred

* how do we make predict better, how do make tune the weight? Loss function....

In [ ]:
[(yout - ygt)**2 for ygt, yout in zip(ys, ypred)] # mean squared error

In [ ]:
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])# mean squared error
loss

* In this case our goal is minimize the loss. Because if loss is low, every one of them predictions are equal to target

In [ ]:
loss.backward()

In [ ]:
n.layers[0].neurons[0].w[0].grad # check some of them neurons's grad

In [ ]:
draw_dot(loss)

This output so complicated because its has a four forward pass for 4 inputs until calculate loss function. After that backpropagation starting to update grad .

In [ ]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]
  


n = MLP(3, [4,4,1]) 


* now we added 'parameters' function for see nicely parameter

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred  = [n(x) for x in xs]
ypred

* this time output was different from the last one because our weight initialize random...

In [ ]:
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

* our first loss value is 5.080107585426276

In [ ]:
n.layers[0].neurons[0].w[0].data 

In [ ]:
n.layers[0].neurons[0].w[0].grad # grad is 0 because backpropagation doesnt start yet

In [ ]:
loss.backward()

In [ ]:
n.layers[0].neurons[0].w[0].data 

In [ ]:
n.layers[0].neurons[0].w[0].grad 

In [ ]:
for p in n.parameters():
    p.data += -0.01 * p.grad # we want to decrease the loss value, so  use learning rate(-0.01) negative.

In [ ]:
n.layers[0].neurons[0].w[0].data 

* This is to minimize the loss function value by operating in learning rate using gradient values ​​on a single neuron value.
* Lets check loss fucntion with updated data

In [ ]:
ypred= [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

* 4.34035897312631(updated loss) < 5.080107585426276(first loss)
* We reduce the loss value !

* When we continuously run the above processes manually, we can observe that the data value in the neurons changes and approaches the desired value.
* But if you increase the learning rate for get a result faster, in some point you may not get update anymore. This is a trade-off. 

* If LR is too low you gonna take way too long to converge but if its too high all thing gets unstable and your loss value may be explode
* So because of that LR is a hyperparameter


* Now all we have to do is we have to iterate this process

In [ ]:
n = MLP(3, [4,4,1]) 
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
ypred  = [n(x) for x in xs]
ypred

In [ ]:
# Gradient DEscent
for step in range(20):
    # forward pass
    ypred= [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    
    # backward pass
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad

    print(step, loss.data)


* We can observe the decrease of the loss value

In [ ]:
ypred

* BUT THERE IS A BUG ! ... 
* When after backpropagation we dont flush the grad. So when we do the second forward pass and backward again these grads just add up never get reset to zero
* *        for p in n.parameters():
                p.data += -0.05 * p.grad # wrong part
* When we have a this bug and gets a good result because our data is so basic so bug has been not effect us . If when we have a more complex problem this bug may being root cause of not optimize the loss  
* Lets fix

In [ ]:
n = MLP(3, [4,4,1]) 

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 


In [ ]:
# Gradient DEscent
for step in range(20):
    # forward pass
    ypred= [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    
    # backward pass
    for p in n.parameters():
        p.grad = 0.0 # now we set grads to zero before backward 
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad

    print(step, loss.data)


In [ ]:
ypred

# The END